# The Fast Fourier Transform

## Introduction

In this notebook/document I explore one of the most important algorithms of the 20th century: __The Fast Fourier Transform__, a revolutionary algorithm that changed the practical application of signal processing and control systems; examples for their use cases include the following (to name a few):

1. De-Noising audio signals
2. Real-time Image and Audio compression
3. Global Communication Networks

## Motivation

My motivation to try to grok this ground breaking algorithm ties in the fact that I am a life long learner and I am always interested in learning new algorithms, especially ones that have changed the world. After much introspection, I came to the conclusion that I don't know much about algorithms and this submission is the result of that ideology.

## Goals of this Submission

1. Explain the concept of a Fourier Series 
2. Elucidate on the Fourier Transform and Discrete Fourier Transform
3. Implement the Fast Fourier Transform

# What is the Fast Fourier Transform (FFT)?

The Fast Fourier Transform (FFT) is an algorithm that efficiently computes the Discrete Fourier Transform (DFT) of a signal or sequence. Fourier analysis, in general, converts a signal from its base or original domain to the frequency domain and the DFT algorithm does this for a discrete signal or sequence on the infinite domain.

What truly makes FFT a tremendously better than the plain vanilla implementation of DFT is that the asymptotic complexity of the transform is $O(nlog(n))$ vs. $O(n^2)$ of the DFT. This means that for sufficiently large inputs, say the cardinality of which is $n=10^{10}$, the FFT algorithm will have $10^{10} \times log_{10}(10^{10}) \approx 3.34 \times 10^{11}$ operations vs. $10^{10 \times 2} = 10^{20}$ operations. That's a significant computation improvement and therefore, has made it feasible to scale to large data sets.


# The Fourier Series

Fourier Series is an infinite sum of sines and cosines of increasing frequencies that approximate $f(x)$, a __periodic__ and __piece-wise__ smooth function. In particular, if $f(x)$ is an L-periodic function on $[0,L)$, it can be written as the following:

\begin{align}
f(x) = \frac{a_0}{2} + \displaystyle\sum_{k=1}^{\infty} \left( a_k cos\left(\dfrac{2\pi kx}{L}\right) + b_ksin\left(\dfrac{2\pi kx}{L} \right)\right)
\end{align}

Where: <br>
\begin{align} a_k = \dfrac{2}{L}\displaystyle\int_{0}^{L} f(x)cos\left(\dfrac{2\pi kx}{L}\right)dx \\
b_k = \dfrac{2}{L}\displaystyle\int_{0}^{L}f(x)sin\left(\dfrac{2\pi kx}{L}\right)dx
\end{align}

Using Euler's Law: 
$e^{ikx} = cos(kx) + isin(kx)$, we can write the Fourier Series equation in __Complex Form__: <br>

\begin{align}
f(x) = \displaystyle\sum_{k=-\infty}^{\infty} c_ke^{ik \pi \frac{x}{L}}
\end{align}

Where: <br>
\begin{align}
c_k = (\alpha_k + i\beta_k) = \dfrac{1}{2L} \displaystyle\int_{-L}^{L} f(x)e^{-ik\pi\frac{x}{L}}dx
\end{align}

and $i$ is $\sqrt{-1}$. This defines a frequency $\omega_k = \dfrac{k\pi}{L}$ that can be used instead in the formula above.

# The Fourier Transform

The Fourier Transform is a Fourier Series that's defined on an infinite domain without the prerequisite of periodicity.

### Derivation

From the previous section's mathematical formulation of the Fourier Series, we have:

\begin{align}
f(x) = \displaystyle\sum_{k=-\infty}^{\infty} c_ke^{ik \pi \frac{x}{L}} = \displaystyle\sum_{k=-\infty}^{\infty} \left( \dfrac{1}{2L} \displaystyle\int_{-L}^{L} f(x)e^{-ik\pi\frac{x}{L}}dx \right) e^{ik\pi \frac{x}{L}}
\end{align}

As the Fourier Transform is defined on an infinite domain, we'll need to adjust the limits of integration to account for that. Specifically, we'll need to take $L \to \infty$ in the equation above. 

However, since, $\omega_k = \dfrac{k\pi}{L}$, we could write $\omega_k = k\Delta\omega$ where $\Delta\omega = \frac{\pi}{L}$ and therefore, $L \to \infty$ could also be written as $\Delta\omega \to 0$. 

Now, writing our $L$ terms as $\Delta\omega$ terms, we get:

- $\frac{1}{2L} = \frac{\Delta\omega}{2\pi}$
- $L = \frac{\pi}{\Delta\omega}$

And this gives us:

\begin{align}
f(x) = \displaystyle\lim_{\Delta\omega \to 0} \displaystyle\sum_{k=-\infty}^{\infty} c_ke^{ik \Delta\omega x} = \displaystyle\lim_{\Delta\omega \to 0} \displaystyle\sum_{k=-\infty}^{\infty}\left(\dfrac{\Delta\omega}{2\pi} \displaystyle\int_{-\frac{\pi}{\Delta\omega}}^{\frac{\pi}{\Delta\omega}} f(\xi)e^{-ik\Delta\omega\xi}d\xi \right) e^{ik \Delta\omega x}
\end{align}

where $\xi$ is a dummy variable used to distinguish $x$ as a separate entity.

The limit above i.e. $\lim_{\Delta\omega \to 0}$, makes the equation of $f(x)$ from an infinite sum into a __Riemman Integral__; we'll consequently be integrating with respect to $d\omega$ giving us:

\begin{align}
 f(x) = \displaystyle\int_{-\infty}^{\infty} \dfrac{1}{2\pi} \left(\displaystyle\int_{-\infty}^{\infty} f(\xi)e^{-i\omega\xi}d\xi\right) e^{i\omega x} d\omega
\end{align}

From above, write the __Fourier Transform__ equation:

\begin{align}
\hat{f}(\omega) = \mathbf{F}(f(x)) = \displaystyle\int_{-\infty}^{\infty} f(x)e^{-i\omega x}dx
\end{align}

And similarly, we can get back to $f(x)$ using the __Inverse-Fourier Transform__ equation: <br>

\begin{align}
f(x) = \mathbf{F}^{-1}(\hat{f}(\omega)) = \dfrac{1}{2\pi} \displaystyle\int_{-\infty}^{\infty} \hat{f}(\omega) e^{i\omega x} d\omega
\end{align}

The __Fourier Transform__ and __Inverse-Fourier Transform__ equations are collectively known as the __Fourier-Transform pair__.

# The Discrete Fourier Transform (DFT)

Since computers represent data in discrete chunks (more specifically, bits that are discrete), to practically compute the Fourier Transform via computers, we need to make use of the discrete flavor of the Fourier Transform. The DFT algorithm essentially takes in a data vector and turns its individual points into its sine and cosine components or is a Fourier Series on data vectors. The Fourier Transform vector, the output of the DFT, is a vector of coefficients corresponding to the successively increasing frequencies associated with the input data vector, in, for example, the time domain.

Mathematically, $\begin{bmatrix} \hat{f}_0 & \hat{f}_1 & \hat{f}_2 & \hat{f}_3 & ... & \hat{f}_n \end{bmatrix}^T$ is obtained from $\begin{bmatrix} f_0 & f_1 & f_2 & f_3 & ... & f_n \end{bmatrix}^T$ <br>

Where:
\begin{align}
\hat{f}_k = \displaystyle \sum_{n = 0}^{N - 1} f_n e^{-i 2\pi n \frac{k}{N}}
\end{align}

Similarly, 
$\begin{bmatrix} f_0 & f_1 & f_2 & f_3 & ... & f_n \end{bmatrix}^T$ can be obtained from $\begin{bmatrix} \hat{f}_0 & \hat{f}_1 & \hat{f}_2 & \hat{f}_3 & ... & \hat{f}_n \end{bmatrix}^T$. <br>

Where:
\begin{align}
f_k = \dfrac{1}{N} \displaystyle\sum_{n=0}^{N-1}\hat{f}_n e^{i2\pi n \frac{k}{N}}
\end{align}

The DFT, thus, can be thought of as the matrix-vector dot product of some input data vector, $\vec{f}$ and a matrix of multiples of fundamental frequencies that are given by $\omega_n = e^{\frac{-2 \pi i}{n}}$.

\begin{align}
\begin{bmatrix} \hat{f}_0 & \hat{f}_1 & \hat{f}_2 & \hat{f}_3 & ... & \hat{f}_n \end{bmatrix}^T = \begin{bmatrix} 1 & 1 & 1 & .. & 1 \\ 1 & \omega_n & \omega_n^2 & .. &  \omega_n^{n - 1} \\ 1 & \omega_n^2 & \omega_n^4 & .. & \omega_n^{2(n - 1)} \\ .. & .. & .. & .. & .. \\ 1 & \omega_n^{n - 1} & \omega_n^{2(n - 1)} & .. & \omega_n^{(n - 1)^2} \end{bmatrix} \begin{bmatrix} f_0 & f_1 & f_2 & f_3 & ... & f_n \end{bmatrix}^T
\end{align}

#### Notes
1. For each, $\hat{f}_k$ we get both the magnitude as well as the phase that sufficiently describes the frequency.
2. The matrix consisting of multiples of fundamental frequencies is called the __DTF Matrix__. 
3. A single $\hat{f}_k$ is an $O(n)$ operation since we are summing over all $n$; we do this $n$ times giving us an asymptotic complexity of $O(n^2)$.

## Slow Implementation of the DFT Matrix

Here I will present two examples of a slow implementation of the DFT Matrix, one with 2 for loops where the first one iterates over the number of rows and the second that iterates over the individual elements i.e. of the $i^{th}$ row and the $j^{th}$ column; the other implementation will be using a List Comprehension.

In general, for the $i^{th}$ row and $j^{th}$ column, the formula for the DFT Matrix can be thought of as:

$\omega_{i,j} = \omega^{(i - 1)(j - 1)}$

In [44]:
import cmath

def create_dftmatrix_slow_forloops(n):
    dft_matrix = []
    ω = cmath.exp(-2j * cmath.pi / n)
    for i in range(1, n + 1):
        row = []
        for j in range(1, n + 1):
            row.append(ω ** ((i - 1)*(j - 1)))
        dft_matrix.append(row)
    return dft_matrix

def create_dftmatrix_slow_listcomprehensions(n):
    ω = cmath.exp(-2j * cmath.pi / n)
    return [[ω ** ((i - 1) * (j - 1)) for j in range(1, n+1)] for i in range(1,n+1)]

# Check for timing.
print("Timing of the DFT Matrix Creation using For Loops: ")
%timeit create_dftmatrix_slow_forloops(10)

print("Timing of the DFT Matrix Creation using List Comprehensions: ")
%timeit create_dftmatrix_slow_listcomprehensions(10)

# Check for equality.
dftmatrix_forloop           = create_dftmatrix_slow_forloops(10)
dftmatrix_listcomprehension = create_dftmatrix_slow_listcomprehensions(10)
assert dftmatrix_forloop == dftmatrix_listcomprehension
print ("dftmatrix_forloop == dftmatrix_listcomprehension")

Timing of the DFT Matrix Creation using For Loops: 
12.6 µs ± 10.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Timing of the DFT Matrix Creation using List Comprehensions: 
11.6 µs ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
dftmatrix_forloop == dftmatrix_listcomprehension


#### Implementation of the DFT Matrix Using Numpy

In [70]:
import numpy as np

def create_dftmatrix_np_slow(n):
    ω             = np.exp(-2j * np.pi / n)
    rows, columns = np.meshgrid(np.arange(n), np.arange(n))
    dft_matrix    = np.power(ω, rows * columns)
    dft_matrix    = np.real(dft_matrix)
    return dft_matrix

def create_dftmatrix_np_fast(n):
    ω          = np.exp(-2j * np.pi / n)
    n_range    = np.arange(n)
    k          = n_range.reshape((n, 1))
    dft_matrix = np.exp(-2j * np.pi * k * n_range / n)
    return dft_matrix

# Check for timing
print("Timing of the Slower Numpy DFT Matrix")
%timeit create_dftmatrix_np_slow(10)

print("Timing of the Faster Numpy DFT Matrix")
%timeit create_dftmatrix_np_fast(10)

# Check for equality
dft_matrix_np_slow = create_dftmatrix_np(10)
dft_matrix_np_fast = create_dftmatrix_np_fast(10)
assert dft_matrix_np_slow.all() == dft_matrix_np_fast.all()
print ("dft_matrix_np_slow == dft_matrix_np_fast")

Timing of the Slower Numpy DFT Matrix
26.1 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Timing of the Faster Numpy DFT Matrix
9.69 µs ± 27.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
dft_matrix_np_slow == dft_matrix_np_fast


#### Computing the DFT

In [66]:
import numpy as np
import cmath

# Source: https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/
# I couldn't have written this algorithm better.
def dft(input_data):
    input_copy = np.asarray(input_data, dtype=float)
    N = input_copy.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, input_copy)

input_data = np.random.random(1024)
%timeit dft(input_data)

82.1 ms ± 77.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Let's compare this to the ``fft`` algorithm in the numpy package.

In [69]:
%timeit np.fft.fft(input_data)

9.03 µs ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


It's obvious that the fft implementation from numpy is __significantly__ faster than the naive implementation above. Let's implement it ourselves!

# Implementation of the Fast Fourier Transform

## Exploiting The Symmetry 

Using Cooley and Tukey's exploitation of the parity based symmetry associated with the DFT formula, we will show how to go from a $O(n^2)$ algorithm to a $O(nlog(n))$ one. Once the symmetery between the even and odd components of the Fourier term is established, we'll use a Divide and Conquer method of solving for the transform.

### Prerequistes

1. Let's first start off with $\hat{f}_{n+k}$ and prove that it's equivalent to $\hat{f}_{k}$

\begin{align}
\hat{f}_{n + k} = \displaystyle \sum_{n = 0}^{N-1} f_n e^{-i 2 \pi (N + k) n/N} \\
\Rightarrow \hat{f}_{n + k} = \displaystyle \sum_{n = 0}^{N-1} f_n e^{-i 2 \pi (Nn + kn)/N} \\
\Rightarrow \hat{f}_{n + k} = \displaystyle \sum_{n = 0}^{N-1} f_n e^{-i 2 \pi (n + kn)/N} \\
\Rightarrow \hat{f}_{n + k} = \displaystyle \sum_{n = 0}^{N-1} f_n e^{-i 2 \pi n(1 + k)/N} \\
\Rightarrow \hat{f}_{n + k} = \displaystyle \sum_{n = 0}^{N-1} f_n e^{-i 2 \pi n} e^{ -i 2 \pi k/N} \\
\end{align}

Now, $e^{-i 2 \pi n} = 1$ as $e^{-i 2 \pi n}$ = $cos(-2n\pi) + isin(-2n\pi) = 1 + 0 = 1$. 

\begin{align}
\Rightarrow \hat{f}_{n + k} = \displaystyle \sum_{n = 0}^{N-1} x_n e^{ -i 2 \pi k/N} = \hat{f}_{k}
\end{align}

And therefore, to generalize:

\begin{align}
\hat{f}_{k + aN} = \hat{f}_{k} \forall a \in \mathbb N
\end{align}

2. Any even number can be written as $2n$ while any odd number can be written as $2n + 1$ where $n \in \mathbb{N}$.

3. A sum can be represented as a sum of odd and even numbers:

\begin{align}
\displaystyle \sum f(x) = \sum_{m \in 2n + 1} f(x) + \sum_{m \in 2n} f(x) \\
\end{align}


### Breaking Up The DFT Equation into Even and Odd Components

To apply the previously mentioned Divide and Conquer technique, we break up the DFT equation into even and odd components.

\begin{align}
\hat{f}_k = \displaystyle \sum_{n=0}^{N-1} f_n e^{-i 2 \pi k n / N}
\end{align}
\begin{align}
\Rightarrow \hat{f}_k = \displaystyle \sum_{m=0}^{N/2-1} f_{2m} e^{-i 2 \pi k (2m) / N} + \sum_{m=0}^{N/2-1} f_{2m + 1} e^{-i 2 \pi k (2m + 1) / N} \\
\Rightarrow \hat{f}_k = \displaystyle \sum_{m=0}^{N/2-1} f_{2m} e^{-i 2 \pi k (m) / N/2 } + e^{-i 2 \pi k/N} \sum_{m=0}^{N/2-1} f_{2m + 1} e^{-i 2 \pi k (m) / N/2}
\end{align}

Here, $\displaystyle \sum_{m=0}^{N/2-1} f_{2m} e^{-i 2 \pi k (m) / N/2 }$ represents the __Even__ component. 

And, $\displaystyle e^{-i 2 \pi k/N} \sum_{m=0}^{N/2-1} f_{2m + 1} e^{-i 2 \pi k (m) / N/2}$ represents the __Odd__ component times a factor.

To elucidate on this a bit more, the Even component only filters on the even-indexed elements of the input vector while the Odd component only filters on the odd-indexed elements. 

In [107]:
import numpy as np

# Code adapted from: https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/
def fft(input_data):
    
    # Nested function to handle the single DFT case.
    def DFT_Single(input_vector):
        input_vector = np.asarray(input_data, dtype = 'float')
        N            = input_data.shape[0] # of Rows of the vector.
        n            = np.arange(N) # Grabs the range from 0 to N - 1
        k            = n.reshape((N, 1)) # Change the shape to fit the bill of a vector
        Ω            = np.exp(-2j * np.pi * k * n / N) # DFT Matrix
        return np.dot(Ω, input_vector)
    
    input_data = np.asarray(input_data, dtype = 'float')
    N          = input_data.shape[0]
    
    # Base case: Compute the Single Case DFT.
    if N <= 32:
        return DFT_Single(input_data)
    # Recursive case.
    else:
        # Recursively apply fft to the even indexed input data.
        f_even   = fft(input_data[::2])
        # Recursively apply fft to the odd indexed input data.
        f_odd    = fft(input_data[1::2])
        # Compute the factor that's multiplied to the odd component.
        factor   = np.exp(-2j * np.pi * np.arange(N) / N)
        # Combine the divided and conquered components split on N/2 so that they combine 
        # to be 'N' cardinality.
        combined = np.concatenate([f_even + factor[:N//2] * f_odd, 
                                  f_even + factor[N//2:] * f_odd])
        return combined

### Checks for Timing and Correctness

In [113]:
# Check timing.
input_hat = np.random.random(2048)
%timeit FFT(input_hat)
%timeit dft(input_hat)

# Check correctness.
print(f'Do the implementations of fft match: \
{np.allclose(fft(input_hat), np.fft.fft(input_hat))}')

3.93 ms ± 38.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
176 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Do the implementations of fft match: True


### Notes

1. Our FFT implementation, as expected, is __significantly__ faster than the naive DFT implementation. 

2. We can further improve this algorithm by including vectorization, but, I'll leave that as an exercise to the reader as that implementation does break the beauty of a simple DAC algorithm.

3. The implementation above follows the general pattern of Divide and Conquer (DAC) Algorithms, whose form is:
```
DAC(a, i, j)
{
    // Base case
    if(small(a, i, j))
      return(Solution(a, i, j))
    else 
      m = divide(a, i, j)    // f1(n)
      c1 = DAC(a, i, mid)    // T(n/2)
      c2 = DAC(a, mid+1, j)  // T(n/2)
      d = combine(c1, c2)    // f2(n)
   return(d)
}
```

##### Analogs of FFT to a Generic Case DAC Algorithm

1. The ``small`` step here is the check to see if the size of the input is <= 32.
2. The ``Solution`` step is the naive DFT algorithm.
3. The ``divide`` step is implicit in the recursion on the divided odd and even indices.
4. The ``DAC`` step is the FFT algorithm recursively called on the divided odd and even indicies.
5. The ``combine`` step is the ``np.concatenate`` on the Fourier Transforms for the even + factor * odd components for [0:N//2] and [N//2:] respectively. These represent the combination of the split up chunks for the N cardinality at each step.

# Summary

1. __Fourier Series__: Allow us to approximate any __periodic__ and __piece-wise smooth__ function via in infinite sum of increasing sin and cosine functions.
2. __Fourier Transforms__: Takes the idea of Fourier Series and allows us to approximate a function as in infinite sum of increasing sin and cosine functions on an infinite domain where the functions to be represented don't necessarily have to be periodic.
3. __Discrete Fourier Transforms (DFT)__: The input to this algorithm is a discrete data vector that is then converted into it's Fourier Representation and the output is a vector of coefficient of the corresponding frequencies. The plain vanilla implementation of this transform is a matrix-vector multiplication of a matrix of fundamental frequencies and the input data vector, respectively. Naively, the asymptotic complexity of this algorithm is $O(n^2)$.
4. __Fast Fourier Transform (FFT)__: A more computationally efficient algorithm to compute the DFT. The asymptotic complexity of this algorithm is $O(nlog(n))$. The premise behind the algorithm is to use the divide-and-conquer technique on the odd and even components of the other-wise large matrix-vector multiplication.

# Resources

1. [Good synopsis from Princeton](https://www.cs.princeton.edu/~wayne/teaching/fft.pdf)
2. [Youtube Playlist for one of the best resources from UWashington](https://www.youtube.com/playlist?list=PLMrJAkhIeNNT_Xh3Oy0Y4LTj0Oxo8GqsC)
3. [Excellent Python Implementation](https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/)

In [114]:
%load_ext watermark
%watermark -n -u -v -iv -w

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
numpy 1.19.2
last updated: Sun Dec 27 2020 

CPython 3.8.5
IPython 7.18.1
watermark 2.0.2
